<a target="_blank" href="https://colab.research.google.com/github/castorgit/DL-course/blob/main/10_RNN_Translator.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Sequence-to-Sequence (Seq2Seq) Machine Translation with RNNs

This Jupyter Notebook implements a **Sequence-to-Sequence (Seq2Seq)** model using Recurrent Neural Networks (RNNs) in Keras and TensorFlow to perform **English-to-Spanish Neural Machine Translation (NMT)**.

The Seq2Seq architecture is a powerful paradigm for tasks involving mapping an input sequence to an output sequence, where the length of the output is not necessarily related to the length of the input. 
## Key Components and Methodology

1.  **Dataset:** The model is trained on a publicly available **English-Spanish parallel corpus**, which is downloaded and processed at the start of the notebook.
2.  **Data Preparation:**
    * Target Spanish sentences are pre-processed by adding `[start]` and `[end]` tokens to mark the boundaries of the sequence.
    * **TextVectorization** layers are used to standardize the text and map words to integer token IDs.
    * The target sequences are split to create input (shifted left by one timestep) and output (original target) sequences, which is the standard setup for **Teacher Forcing** during training.
3.  **Model Architecture (Encoder-Decoder RNN):**
    * **Encoder:** A **Bidirectional GRU** (Gated Recurrent Unit) processes the source (English) sentence, capturing context from both directions. The final hidden state of the encoder serves as the initial state (context vector) for the decoder.
    * **Decoder:** A **GRU** layer takes the target (Spanish) input sequence (preceded by `[start]`) and is initialized with the encoder's context vector. It predicts the next word in the sequence at each timestep.
4.  **Inference (Decoding):** A **greedy sampling** approach is implemented to translate new English sentences. The model generates the target sequence one word at a time, feeding its own prediction back as input for the next step until the `[end]` token is generated.

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2025-12-15 09:15:30--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.184.27, 142.250.185.27, 142.250.200.123, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.184.27|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  3.67MB/s    in 0.7s    

2025-12-15 09:15:31 (3.67 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]

/bin/bash: line 1: unzip: command not found


In [1]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [2]:
import tensorflow as tf
print(tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    details = tf.config.experimental.get_device_details(gpus[0])
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(details.get("device_name", "Unknown GPU"))
else:
    print("No GPU detected")

2025-12-15 10:36:43.531725: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 10:36:43.840368: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/jaumemanero/RL/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_c

2.20.0
NVIDIA GeForce RTX 3090


In [3]:
import random
print(random.choice(text_pairs))

('Tom should have made up his mind by now.', '[start] Tom ya debería haber tomado una decisión. [end]')


In [4]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [5]:
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

I0000 00:00:1765791407.655357     721 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21770 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:52:00.0, compute capability: 8.6


In [6]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [7]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


2025-12-15 10:36:48.560636: W tensorflow/core/kernels/data/cache_dataset_ops.cc:917] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-12-15 10:36:48.561518: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

In [9]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [10]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15


2025-12-15 10:36:51.535189: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


1302/1302 ━━━━━━━━━━━━━━━━━━━━ 128s 96ms/step - accuracy: 0.3200 - loss: 4.6684 - val_accuracy: 0.3897 - val_loss: 3.8772
Epoch 2/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 124s 95ms/step - accuracy: 0.4151 - loss: 3.7240 - val_accuracy: 0.4664 - val_loss: 3.2508
Epoch 3/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 121s 93ms/step - accuracy: 0.4715 - loss: 3.2214 - val_accuracy: 0.5135 - val_loss: 2.8826
Epoch 4/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 123s 95ms/step - accuracy: 0.5115 - loss: 2.8705 - val_accuracy: 0.5488 - val_loss: 2.6301
Epoch 5/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 121s 93ms/step - accuracy: 0.5446 - loss: 2.5989 - val_accuracy: 0.5763 - val_loss: 2.4478
Epoch 6/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 124s 95ms/step - accuracy: 0.5735 - loss: 2.3741 - val_accuracy: 0.5949 - val_loss: 2.3242
Epoch 7/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 121s 93ms/step - accuracy: 0.5977 - loss: 2.1895 - val_accuracy: 0.6117 - val_loss: 2.2207
Epoch 8/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 124s 95ms/step - accuracy: 0.6190 - lo

In [11]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
The young girl wanted to be a star of the silver screen.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
[start] la chica ni una chica [UNK] como la [UNK] de la [UNK] son [UNK] [end]
-
Leave a blank space.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
[start] deja una moneda de aire [end]
-
I

In [12]:
import session_info
session_info.show(html=False)

-----
keras               3.12.0
numpy               2.2.6
session_info        v1.0.1
tensorflow          2.20.0
-----
IPython             8.37.0
jupyter_client      8.6.3
jupyter_core        5.8.1
jupyterlab          4.4.5
notebook            7.4.4
-----
Python 3.10.12 (main, Nov  4 2025, 08:48:33) [GCC 11.4.0]
Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35
-----
Session information updated at 2025-12-15 11:10
